# Slackbot für die nationalen Wahlen
Dieses Script liefert dem Züri-Ressort des TA während dem Wahlsonntag aktuelle Resultate und Hochrechnungen. Ziel ist es, das Team in der Live-Berichterstattung zu unterstützen. Neue Resultate gehen weniger unter und können schnelle eingeordnet werden. Damit das Script wiederholt abgespielt wird, liegt es auf einem server und wird von einem Cronjob aktiviert.

In [ ]:
import pandas as pd
import requests
import time 
import datetime
import json
import numpy
import re
import os
import slack
from slack_api_token import SLACK_API_TOKEN
from stat_api_urls import url_nr_hoch
from stat_api_urls import url_nr_resultat
from stat_api_urls import url_nr_resultat_kandidierende
from stat_api_urls import url_sr_hoch
from stat_api_urls import url_sr_resultat_gemeinde
from stat_api_urls import url_sr_resultat

In [2]:
#client = slack.WebClient(token=SLACK_API_TOKEN)
client = slack.WebClient(
    token=SLACK_API_TOKEN,
    run_async=True
    )

# Nationalrat Hochrechnungen
Sende Updates, wenn neue Hochrechnung vorliegt (Personen oder Parteien)

In [5]:
response = requests.get(url_nr_hoch)
data = response.json()

## Parteien

In [6]:
time.sleep(1)
lastupdate = open("hr_update_nr_parteien.txt","r").read()

In [7]:
data_available = len(data['hochrechnung_parteien']['hochrechnung_fuer_parteien'])

In [8]:
if data['hochrechnung_parteien']['hochrechnung_fuer_parteien_metadaten']['zeitpunkt_der_hochrechnung'] != lastupdate and data_available>0:
    hochrechnung_nr_zeitpunkt = data['hochrechnung_parteien']['hochrechnung_fuer_parteien_metadaten']['zeitpunkt_der_hochrechnung']
    hochrechnung_nr_titel = "Nationalrat (Parteien): " + hochrechnung_nr_zeitpunkt
    hochrechnung_nr_titel
    hoch_parteien = data['hochrechnung_parteien']['hochrechnung_fuer_parteien']
    hoch_nr_text = str()
    for partei in hoch_parteien:
        partei_nr_text = "*"+str(partei['parteicode']) + "*: " + str(partei['waehlerproz_2019']) + "% (" +  str(partei['waehlerproz_delta']) + ") und " + str(partei['sitze_2019']) + " Sitz(e) (" +  str(partei['sitze_delta']) + ") \n "
        hoch_nr_text = hoch_nr_text + partei_nr_text 
    hoch_nr_text = "*```"+hochrechnung_nr_titel+"```* \n" + hoch_nr_text
    hoch_nr_text = hoch_nr_text.replace('.',',')
    #send-message
    response = client.chat_postMessage(
        channel='#wahlen19-zh-results',
        text=hoch_nr_text)
    #save hochrechnungszeitpunkt
    updatefile = open("hr_update_nr_parteien.txt","w")#write mode 
    updatefile.write(hochrechnung_nr_zeitpunkt) 
    updatefile.close() 

else:
    print('no news')

no news


## Personen

In [9]:
time.sleep(1)
lastupdate = open("hr_update_nr_personen.txt","r").read()
lastupdate

'12:10'

In [10]:
data_available = len(data['hochrechnung_kandidaten']['hochrechnung_fuer_kandidaten'])

In [11]:
if data['hochrechnung_kandidaten']['hochrechnung_fuer_kandidaten_metadaten']['zeitpunkt_der_hochrechnung'] != lastupdate and data_available>0:
    hochrechnung_nr_personen_zeitpunkt = data['hochrechnung_kandidaten']['hochrechnung_fuer_kandidaten_metadaten']['zeitpunkt_der_hochrechnung']
    hochrechnung_nr_personen_titel = "Nationalrat (Personen): Hochrechnung von " + hochrechnung_nr_personen_zeitpunkt
    hochrechnung_nr_personen = data['hochrechnung_kandidaten']['hochrechnung_fuer_kandidaten']
    #abgewählte, neugewählte
    df_hr_nr_personen = pd.DataFrame(hochrechnung_nr_personen)
    df_abgewaehlt = df_hr_nr_personen[(df_hr_nr_personen['bisher']==True) & (df_hr_nr_personen['mutmasslich_gewaehlt']==False)]
    df_neugewaehlt = df_hr_nr_personen[(df_hr_nr_personen['bisher']==False) & (df_hr_nr_personen['mutmasslich_gewaehlt']==True)]
    neugewaehlte = df_neugewaehlt['vorname'] + " " + df_neugewaehlt['name']+ " (" + df_neugewaehlt['parteicode'] +") "
    abgewaehlte = df_abgewaehlt['vorname'] + " " + df_abgewaehlt['name']+ " (" + df_abgewaehlt['parteicode'] +") "
    neugewaehlte = neugewaehlte.tolist()
    abgewaehlte = abgewaehlte.tolist()
    kandidaten_nr_einleitung = str(len(abgewaehlte))+" Kandidierende müssen zittern, " + str(len(neugewaehlte)) + " dürfen hoffen: \n\n"
    kandidaten_nr_neu_text = str()
    for neugewaehlt in neugewaehlte:
        kandidaten_nr_neu_text = kandidaten_nr_neu_text+ neugewaehlt + " \n "
    kandidaten_nr_abg_text = str()
    for abgewaehlt in abgewaehlte:
        kandidaten_nr_abg_text = kandidaten_nr_abg_text+ abgewaehlt + " \n "
    hoch_nr_personen_text = "*```"+hochrechnung_nr_personen_titel+"```*"+ "\n" + kandidaten_nr_einleitung + "*Laut Hochrechnung abgewählt:* \n " + kandidaten_nr_abg_text + "\n *Laut Hochrechnung neu im Parlament:* \n " + kandidaten_nr_neu_text
    #send-message
    response = client.chat_postMessage(
        channel='#wahlen19-zh-results',
        text=hoch_nr_personen_text)
    #save hochrechnungszeitpunkt
    updatefile = open("hr_update_nr_personen.txt","w")#write mode 
    updatefile.write(hochrechnung_nr_personen_zeitpunkt) 
    updatefile.close() 
else:
    print('no news')
    

no news


# Nationalrat Resultate
Sende Updates, wenn 10 zusätzliche Gemeinden ausgezählt sind oder das Schlussresultat

In [12]:
time.sleep(1)

In [13]:
response = requests.get(url_nr_resultat)
data = response.json()

In [14]:
schlussresultat = data['metadaten']['stand']['wahl_abgeschlossen']

In [15]:
gebiete_ausgezaehlt = str(data['metadaten']['stand']['stand_kantonsebene_gebiete']['gebiete_abgeschlossen'])
gebiete_total = str(data['metadaten']['stand']['stand_kantonsebene_gebiete']['gebiete_total'])

In [16]:
resultate_parteien = data['liste_partei_auf_kantonsebene']['partei_auf_kantonsebene']
#Nur ausgewählte Parteien
parteiliste = ['SVP','SP','FDP','glp','CVP','GRÜNE','BDP','EVP','AL']

In [17]:
#Differenz zum letzten Auszählungsstand ausrechnen.
lastupdate = open("update_nr_parteien_muss_null_sein_am_anfang.txt","r").read()
lastupdate = int(gebiete_ausgezaehlt) - int(lastupdate)
done = open("sr_fertig.txt","r").read()

In [18]:
if lastupdate > 9 and schlussresultat == False:
    trend_text = str()
    for resultat in resultate_parteien:
        if resultat['partei_bezeichnung'] in parteiliste:
            satz = resultat['partei_bezeichnung'] + ": "
            if resultat['differenz_partei_staerke']>2.5:
                satz = satz + "Liegt deutlich im Plus. (" + str(resultat['differenz_partei_staerke']) +") \n"
            elif resultat['differenz_partei_staerke']>0.2:
                satz = satz + "Liegt im Plus. (" + str(resultat['differenz_partei_staerke']) +") \n"
            elif resultat['differenz_partei_staerke']<0.2 and resultat['differenz_partei_staerke']>-0.2:
                satz = satz + "Bleibt relativ konstant. (" + str(resultat['differenz_partei_staerke']) +") \n"
            elif resultat['differenz_partei_staerke']<-2.5:
                satz = satz + "Wähleranteil sinkt deutlich. (" + str(resultat['differenz_partei_staerke']) +") \n"
            elif resultat['differenz_partei_staerke']<-0.2:
                satz = satz + "Wähleranteil sinkt. (" + str(resultat['differenz_partei_staerke']) +") \n"
            trend_text = trend_text + satz
        else:
            continue
    trend_text = "*```Trendrechnung Nationalrat:```*\n Stand nach " + str(gebiete_ausgezaehlt) + " von " + str(gebiete_total) + " ausgezählten Gebieten. (Vergleich zu 2015)\n" + trend_text
    #send-message
    response = client.chat_postMessage(
        channel='#wahlen19-zh-results',
        text=trend_text)
    #save auszählungsstand
    updatefile = open("update_nr_parteien_muss_null_sein_am_anfang.txt","w")#write mode 
    updatefile.write(gebiete_ausgezaehlt) 
    updatefile.close() 
else:
    print('no news')

no news


In [19]:
if schlussresultat == True and done!="True":
    trend_text = str()
    for resultat in resultate_parteien:
        if resultat['partei_bezeichnung'] in parteiliste:
            satz = resultat['partei_bezeichnung'] + ": "
            if resultat['differenz_partei_staerke']>2:
                satz = satz + "Legt deutlich zu. " + str(resultat['partei_staerke']) + "% (" + str(resultat['differenz_partei_staerke']) + ")"+ " | " + str(resultat['anz_gewaehlte'])+ " Sitze / " + str(resultat['differenz_anz_gewaehlte'])+" \n"
            elif resultat['differenz_partei_staerke']>0.3:
                satz = satz + "Legt zu. " + str(resultat['partei_staerke']) + "% (" + str(resultat['differenz_partei_staerke']) + ")"+ " | " + str(resultat['anz_gewaehlte'])+ " Sitze / " + str(resultat['differenz_anz_gewaehlte'])+" \n"
            elif resultat['differenz_partei_staerke']<0.3 and resultat['differenz_partei_staerke']>-0.3:
                satz = satz + "Bleibt relativ konstant. " + str(resultat['partei_staerke']) + "% (" + str(resultat['differenz_partei_staerke']) + ")"+ " | " + str(resultat['anz_gewaehlte'])+ " Sitze / " + str(resultat['differenz_anz_gewaehlte'])+" \n"
            elif resultat['differenz_partei_staerke']<-2:
                satz = satz + "Wähleranteil sinkt deutlich. " + str(resultat['partei_staerke']) + "(" + str(resultat['differenz_partei_staerke']) + ")"+ " | " + str(resultat['anz_gewaehlte'])+ " Sitze / " + str(resultat['differenz_anz_gewaehlte'])+" \n"
            elif resultat['differenz_partei_staerke']<-0.3:
                satz = satz + "Wähleranteil sinkt. " + str(resultat['partei_staerke']) + "% (" + str(resultat['differenz_partei_staerke']) + ")"+ " | " + str(resultat['anz_gewaehlte'])+ " Sitze / " + str(resultat['differenz_anz_gewaehlte'])+" \n"
            trend_text = trend_text + satz
        else:
            continue
    trend_text = "*```Schlussresultat Nationalrat:```*\n\n " + trend_text
    #send-message
    response = client.chat_postMessage(
        channel='#wahlen19-zh-results',
        text=trend_text)
    # finde die neu- und abgewählten und schicke das gleich auch noch
    kandidierende = url_nr_resultat_kandidierende
    response = requests.get(kandidierende)
    data = response.json()
    df = pd.DataFrame(data['kandidaten'])
    df['liste_bezeichnung'] = df.liste_bezeichnung.str.replace('glp','GLP')
    df['liste_bezeichnung'] = df.liste_bezeichnung.str.replace('GRÜNE','Grüne')
    df['namen'] = df['vorname'] + " " + df['name'] + " (" + df['liste_bezeichnung'] +")"
    neugewaehlt = df[['namen']][(df['flag_gewaehlt']==True) & (df['bisher']==False)].to_string()
    abgewaehlt = df[['namen']][(df['flag_gewaehlt']==False) & (df['bisher']==True)].to_string()
    response = client.chat_postMessage(
            channel='#wahlen19-zh-results',
            text="*```Das sind die Neuen im Nationalrat```*: \n" + neugewaehlt)
    response = client.chat_postMessage(
        channel='#wahlen19-zh-results',
        text="*```Sie haben die Wiederwahl verpasst: ```*: \n" + abgewaehlt)
    #save auszählungsstand
    updatefile = open("update_nr_parteien_muss_null_sein_am_anfang.txt","w")#write mode 
    updatefile.write(gebiete_ausgezaehlt) 
    updatefile.close()
    updatefile = open("nr_fertig.txt","w")#write mode 
    updatefile.write(str(schlussresultat)) 
    updatefile.close()
else:
    print('not finished yet')

not finished yet


# Ständerat Hochrechnungen

In [20]:
#Daten Hochrechnung
time.sleep(1)
response = requests.get(url_sr_hoch)
data = response.json()

In [21]:
hoch_sr_zeitpunkt = data['hochrechnung_fuer_kandidaten_metadaten']['zeitpunkt_der_hochrechnung']
lastupdate = open("hr_update_sr_personen.txt","r").read()

In [22]:
data_available = len(data['hochrechnung_fuer_kandidaten'])

In [23]:
hochrechnung_sr_text = str()
if hoch_sr_zeitpunkt != lastupdate and data_available > 0:
    abs_mehr_upper = data['hochrechnung_fuer_absolutes_mehr']['vtupper']
    abs_mehr_lower = data['hochrechnung_fuer_absolutes_mehr']['vtlower']
    abs_mehr = data['hochrechnung_fuer_absolutes_mehr']['stimmenzahl']
    kandidaten = data['hochrechnung_fuer_kandidaten']
    kandidaten = sorted(kandidaten, key = lambda i: i['stimmenzahl'], reverse=True) 
    hochrechnung_sr_text = "*```Hochrechnung Ständerat: "+ hoch_sr_zeitpunkt +"```*\n"

    for kandidat in kandidaten:
        
        name = kandidat['vorname'] + " " + kandidat['name']
        stimmenzahl = str(kandidat['stimmenzahl'] * 1000)
        lower = kandidat['vtlower']
        upper = kandidat['vtupper']
        absmehrsatz = 'Die Prognose hat noch deutliche Unsicherheiten.'
        if lower > abs_mehr_upper:
            absmehrsatz = 'Das reicht für das absolute Mehr. :white_check_mark:'
        elif lower > abs_mehr:
            absmehrsatz = 'Das reicht wahrscheinlich für das absolute Mehr. :ballo_box_with_check:'
        elif upper < abs_mehr_lower:
            absmehrsatz = 'Absolutes Mehr nicht erreicht. :x:'
        elif upper < abs_mehr:
            absmehrsatz = 'Absolutes wahrscheinlich nicht erreicht. '
        kanidaten_text = "*"+name + "*: " + stimmenzahl + " Stimmen. " + absmehrsatz+"\n\n"
        hochrechnung_sr_text = hochrechnung_sr_text + kanidaten_text
        #send-message
    response = client.chat_postMessage(
        channel='#wahlen19-zh-results',
        text=hochrechnung_sr_text)
    #save hochrechnung update
    updatefile = open("hr_update_sr_personen.txt","w")#write mode 
    updatefile.write(hoch_sr_zeitpunkt) 
    updatefile.close()
else:
    print('no news')

no news


### Eurovision-Song-Contest-Battle

In [24]:
def get_gewinner(row):
    satz = row['namen'] + " gewinnt " + str(row[True]) + " Gebiete."
    return satz

In [25]:
response = requests.get(url_sr_resultat_gemeinde)
data = response.json()
gebiete_abgeschlossen = data['metadaten']['stand']['stand_kantonsebene_gebiete']['gebiete_abgeschlossen']
last_update = open("update_sr_personen_muss_null_sein_am_anfang.txt","r").read()

In [26]:
wait_for_eurovision = gebiete_abgeschlossen - int(last_update)

In [27]:
#Daten Resultate
if wait_for_eurovision > 19 and gebiete_abgeschlossen > 0:
    df_sr_results = pd.DataFrame(data['kandidaten'])
    df_sr_results['namen'] = df_sr_results['vorname'] + " " + df_sr_results['name']
    df = df_sr_results[~df_sr_results['stimmen_kandidat'].isna()]
    #Merge Maximal-Stimmen pro Gemeinde an Datensatz
    df_max = df.merge(df.groupby(['gemeinde_nummer']).stimmen_kandidat.max().reset_index(), on='gemeinde_nummer')
    #Generiere Spalte mit den jeweiligen Gewinnern der Gemeinde
    df_max['gewinnt'] = df_max['stimmen_kandidat_x'] == df_max['stimmen_kandidat_y']
    gewinner = df_max[df_max['gewinnt']==True].groupby('namen').gewinnt.value_counts().unstack().sort_values(True, ascending=False).reset_index()
    saetze = gewinner[['namen',True]].apply(get_gewinner,axis=1).to_string()
    response = client.chat_postMessage(
          channel='#wahlen19-zh-results',
        text="*```Eurovision-Contest des Ständerats```* \n("  + str(gebiete_abgeschlossen) + ") ausgezählt \n" + saetze)
    updatefile = open("update_sr_personen_muss_null_sein_am_anfang.txt","w")#write mode 
    updatefile.write(str(gebiete_abgeschlossen)) 
    updatefile.close()
else:
    print('no news')

no news


# Ständerat Schlussresultat

In [31]:
response = requests.get(url_sr_resultat)
data = response.json()
fertig = data['metadaten']['stand']['wahl_abgeschlossen']
done = open("sr_fertig.txt","r").read()

In [29]:
if fertig == True and done != 'True':
    schluss_sr_text = str()
    kandidaten = data['kandidaten']
    kandidaten = sorted(kandidaten, key = lambda i: i['stimmen_kandidat'], reverse=True) 
    for kandidat in kandidaten:
        name = str(kandidat['vorname']) + " " + str(kandidat['name'])
        stimmenzahl = str(kandidat['stimmen_kandidat'])
        stimmenprozent = str(kandidat['stimmen_prozent'])
        absolutes_mehr = str(kandidat['absolutes_mehr'])
        if kandidat['flag_gewaehlt']==1: 
            kandidaten_text = "*"+name + "*: " + stimmenzahl + " Stimmen. ("+ stimmenprozent+"%)" + ":ballo_box_with_check: \n\n"
        else:
            kandidaten_text = "*"+name + "*: " + stimmenzahl + " Stimmen. ("+ stimmenprozent+"%)" + "\n\n"
        schluss_sr_text = schluss_sr_text + kandidaten_text
    schluss_sr_text = "*```Schlussresultat Ständerat```*\n" + schluss_sr_text
    response = client.chat_postMessage(
          channel='#wahlen19-zh-results',
        text=schluss_sr_text)
else:
    print('not finished / or endresults already sent')

not finished / or endresults already sent


In [30]:
if fertig == True and done != 'True':
    time.sleep(5)
    response = requests.get(url_sr_resultat_gemeinde)
    data = response.json()
    gebiete_abgeschlossen = data['metadaten']['stand']['stand_kantonsebene_gebiete']['gebiete_abgeschlossen']
    df_sr_results = pd.DataFrame(data['kandidaten'])
    df_sr_results['namen'] = df_sr_results['vorname'] + " " + df_sr_results['name']
    df = df_sr_results[~df_sr_results['stimmen_kandidat'].isna()]
    #Merge Maximal-Stimmen pro Gemeinde an Datensatz
    df_max = df.merge(df.groupby(['gemeinde_nummer']).stimmen_kandidat.max().reset_index(), on='gemeinde_nummer')
    #Generiere Spalte mit den jeweiligen Gewinnern der Gemeinde
    df_max['gewinnt'] = df_max['stimmen_kandidat_x'] == df_max['stimmen_kandidat_y']
    gewinner = df_max[df_max['gewinnt']==True].groupby('namen').gewinnt.value_counts().unstack().sort_values(True, ascending=False).reset_index()
    saetze = gewinner[['namen',True]].apply(get_gewinner,axis=1).to_string()
    response = client.chat_postMessage(
          channel='#wahlen19-zh-results',
        text="*```Eurovision-Contest des Ständerats```*\n("  + str(gebiete_abgeschlossen) + " ausgezählt) \n" + saetze)
    updatefile = open("sr_fertig.txt","w")#write mode 
    updatefile.write(str(fertig)) 
    updatefile.close()
else:
    print('not finished / or endresults already sent')

not finished / or endresults already sent
